In [1]:
import numpy
# fix random seed for reproducibility
numpy.random.seed(7)
import pandas as pd
import numpy as np
from __future__ import division, print_function, unicode_literals


import matplotlib
import matplotlib.pyplot as plt

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

tf.reset_default_graph()
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

np.random.seed(42)
tf.set_random_seed(42)

/home/inderjeet78/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data = pd.read_csv("creditcard.csv")
data['normAmount'] = StandardScaler().fit_transform(data['Amount'].values.reshape(-1, 1))
data = data.drop(['Time','Amount'],axis=1)
X = data.iloc[:,data.columns != 'Class']
Y = data.iloc[:,data.columns == 'Class']
pca = PCA(n_components=25)
X = pca.fit_transform(X)
X_train,testX,Y_train,testY = train_test_split(X,Y,test_size=0.20,random_state=21, stratify=Y)
X_train,valX,Y_train,valY = train_test_split(X_train,Y_train,test_size=0.20,random_state=21, stratify=Y_train)

In [3]:
X_train = np.array(X_train)
valX = np.array(valX)
testX = np.array(testX)
Y_train = np.array(Y_train)
valY = np.array(valY)
testY = np.array(testY)

# Graph Prepration

In [4]:
X = tf.placeholder(shape=[None, 5, 5, 1], dtype=tf.float32, name="X")

caps1_n_maps = 8
caps1_n_caps = caps1_n_maps * 4 * 4  # 1152 primary capsules
caps1_n_dims = 4

conv1_params = {
    "filters": 32,
    "kernel_size": 2,
    "strides": 1,
    "padding": "valid",
    "activation": tf.nn.relu,
}

"""conv2_params = {
    "filters": caps1_n_maps * caps1_n_dims, # 256 convolutional filters
    "kernel_size": 9,
    "strides": 2,
    "padding": "valid",
    "activation": tf.nn.relu
}"""

conv1 = tf.layers.conv2d(X, name="conv1", **conv1_params)
#conv2 = tf.layers.conv2d(conv1, name="conv2", **conv2_params)

caps1_raw = tf.reshape(conv1, [-1, caps1_n_caps, caps1_n_dims],
                       name="caps1_raw")

def squash(s, axis=-1, epsilon=1e-7, name=None):
    with tf.name_scope(name, default_name="squash"):
        squared_norm = tf.reduce_sum(tf.square(s), axis=axis,
                                     keep_dims=True)
        safe_norm = tf.sqrt(squared_norm + epsilon)
        squash_factor = squared_norm / (1. + squared_norm)
        unit_vector = s / safe_norm
        return squash_factor * unit_vector

caps1_output = squash(caps1_raw, name="caps1_output")

caps2_n_caps = 2
caps2_n_dims = 6

init_sigma = 0.1

W_init = tf.random_normal(
    shape=(1, caps1_n_caps, caps2_n_caps, caps2_n_dims, caps1_n_dims),
    stddev=init_sigma, dtype=tf.float32, name="W_init")
W = tf.Variable(W_init, name="W")

batch_size = tf.shape(X)[0]
W_tiled = tf.tile(W, [batch_size, 1, 1, 1, 1], name="W_tiled")

caps1_output_expanded = tf.expand_dims(caps1_output, -1,
                                       name="caps1_output_expanded")
caps1_output_tile = tf.expand_dims(caps1_output_expanded, 2,
                                   name="caps1_output_tile")
caps1_output_tiled = tf.tile(caps1_output_tile, [1, 1, caps2_n_caps, 1, 1],
                             name="caps1_output_tiled")

caps2_predicted = tf.matmul(W_tiled, caps1_output_tiled,
                            name="caps2_predicted")

# Dynamic Routing algorithm
# Round 1
raw_weights = tf.zeros([batch_size, caps1_n_caps, caps2_n_caps, 1, 1],
                       dtype=np.float32, name="raw_weights")
routing_weights = tf.nn.softmax(raw_weights, dim=2, name="routing_weights")

weighted_predictions = tf.multiply(routing_weights, caps2_predicted,
                                   name="weighted_predictions")
weighted_sum = tf.reduce_sum(weighted_predictions, axis=1, keep_dims=True,
                             name="weighted_sum")
caps2_output_round_1 = squash(weighted_sum, axis=-2,
                              name="caps2_output_round_1")

caps2_output_round_1_tiled = tf.tile(
    caps2_output_round_1, [1, caps1_n_caps, 1, 1, 1],
    name="caps2_output_round_1_tiled")

agreement1 = tf.matmul(caps2_predicted, caps2_output_round_1_tiled,
                      transpose_a=True, name="agreement1")
# Round 2
# Routing weight update
raw_weights_round_2 = tf.add(raw_weights, agreement1,
                             name="raw_weights_round_2")
routing_weights_round_2 = tf.nn.softmax(raw_weights_round_2,
                                        dim=2,
                                        name="routing_weights_round_2")
weighted_predictions_round_2 = tf.multiply(routing_weights_round_2,
                                           caps2_predicted,
                                           name="weighted_predictions_round_2")
weighted_sum_round_2 = tf.reduce_sum(weighted_predictions_round_2,
                                     axis=1, keep_dims=True,
                                     name="weighted_sum_round_2")
caps2_output_round_2 = squash(weighted_sum_round_2,
                              axis=-2,
                              name="caps2_output_round_2")
caps2_output_round_2_tiled = tf.tile(
    caps2_output_round_2, [1, caps1_n_caps, 1, 1, 1],
    name="caps2_output_round_2_tiled")

agreement2 = tf.matmul(caps2_predicted, caps2_output_round_2_tiled,
                      transpose_a=True, name="agreement2")

# Round 3
# Routing weight update
raw_weights_round_3 = tf.add(raw_weights_round_2, agreement2,
                             name="raw_weights_round_3")
routing_weights_round_3 = tf.nn.softmax(raw_weights_round_3,
                                        dim=2,
                                        name="routing_weights_round_3")
weighted_predictions_round_3 = tf.multiply(routing_weights_round_3,
                                           caps2_predicted,
                                           name="weighted_predictions_round_3")
weighted_sum_round_3 = tf.reduce_sum(weighted_predictions_round_3,
                                     axis=1, keep_dims=True,
                                     name="weighted_sum_round_3")
caps2_output_round_3 = squash(weighted_sum_round_3,
                              axis=-2,
                              name="caps2_output_round_3")
caps2_output_round_3_tiled = tf.tile(
    caps2_output_round_3, [1, caps1_n_caps, 1, 1, 1],
    name="caps2_output_round_3_tiled")

agreement3 = tf.matmul(caps2_predicted, caps2_output_round_3_tiled,
                      transpose_a=True, name="agreement3")

# Round 4
# Routing weight update
raw_weights_round_4 = tf.add(raw_weights_round_3, agreement3,
                             name="raw_weights_round_4")
routing_weights_round_4 = tf.nn.softmax(raw_weights_round_4,
                                        dim=2,
                                        name="routing_weights_round_4")
weighted_predictions_round_4 = tf.multiply(routing_weights_round_4,
                                           caps2_predicted,
                                           name="weighted_predictions_round_4")
weighted_sum_round_4 = tf.reduce_sum(weighted_predictions_round_4,
                                     axis=1, keep_dims=True,
                                     name="weighted_sum_round_4")
caps2_output_round_4 = squash(weighted_sum_round_4,
                              axis=-2,
                              name="caps2_output_round_4")
"""caps2_output_round_4_tiled = tf.tile(
    caps2_output_round_4, [1, caps1_n_caps, 1, 1, 1],
    name="caps2_output_round_4_tiled")

agreement4 = tf.matmul(caps2_predicted, caps2_output_round_4_tiled,
                      transpose_a=True, name="agreement3")
"""


caps2_output = caps2_output_round_4

def safe_norm(s, axis=-1, epsilon=1e-7, keep_dims=False, name=None):
    with tf.name_scope(name, default_name="safe_norm"):
        squared_norm = tf.reduce_sum(tf.square(s), axis=axis,
                                     keep_dims=keep_dims)
        return tf.sqrt(squared_norm + epsilon)

y_proba = safe_norm(caps2_output, axis=-2, name="y_proba")


y_proba_argmax = tf.argmax(y_proba, axis=2, name="y_proba")

y_pred = tf.squeeze(y_proba_argmax, axis=[1,2], name="y_pred")

y = tf.placeholder(shape=[None], dtype=tf.int64, name="y")

m_plus = 0.95
m_minus = 0.05
lambda_ = 100

T = tf.one_hot(y, depth=caps2_n_caps, name="T")

caps2_output_norm = safe_norm(caps2_output, axis=-2, keep_dims=True,
                              name="caps2_output_norm")

present_error_raw = tf.square(tf.maximum(0., m_plus - caps2_output_norm),
                              name="present_error_raw")
present_error = tf.reshape(present_error_raw, shape=(-1, 2),
                           name="present_error")
present_error

absent_error_raw = tf.square(tf.maximum(0., caps2_output_norm - m_minus),
                             name="absent_error_raw")
absent_error = tf.reshape(absent_error_raw, shape=(-1, 2),
                          name="absent_error")

# Loss Function

L = tf.add(T * present_error, lambda_ * (1.0 - T) * absent_error,
           name="L")

margin_loss = tf.reduce_mean(tf.reduce_sum(L, axis=1), name="margin_loss")

mask_with_labels = tf.placeholder_with_default(False, shape=(),
                                               name="mask_with_labels")

reconstruction_targets = tf.cond(mask_with_labels, # condition
                                 lambda: y,        # if True
                                 lambda: y_pred,   # if False
                                 name="reconstruction_targets")

reconstruction_mask = tf.one_hot(reconstruction_targets,
                                 depth=caps2_n_caps,
                                 name="reconstruction_mask")

reconstruction_mask_reshaped = tf.reshape(
    reconstruction_mask, [-1, 1, caps2_n_caps, 1, 1],
    name="reconstruction_mask_reshaped")

caps2_output_masked = tf.multiply(
    caps2_output, reconstruction_mask_reshaped,
    name="caps2_output_masked")

decoder_input = tf.reshape(caps2_output_masked,
                           [-1, caps2_n_caps * caps2_n_dims],
                           name="decoder_input")

n_hidden1 = 16
n_hidden2 = 32
n_output = 5*5

with tf.name_scope("decoder"):
    hidden1 = tf.layers.dense(decoder_input, n_hidden1,
                              activation=tf.nn.relu,
                              name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2,
                              activation=tf.nn.relu,
                              name="hidden2")
    decoder_output = tf.layers.dense(hidden2, n_output,
                                     activation=tf.nn.sigmoid,
                                     name="decoder_output")

X_flat = tf.reshape(X, [-1, n_output], name="X_flat")
squared_difference = tf.square(X_flat - decoder_output,
                               name="squared_difference")
reconstruction_loss = tf.reduce_mean(squared_difference,
                                    name="reconstruction_loss")

alpha = 0.0005

loss = tf.add(margin_loss, alpha * reconstruction_loss, name="loss")


correct = tf.equal(y, y_pred, name="correct")
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

optimizer = tf.train.AdamOptimizer()
training_op = optimizer.minimize(loss, name="training_op")

init = tf.global_variables_initializer()
saver = tf.train.Saver()

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
dim is deprecated, use axis instead


# Training

In [5]:
n_epochs = 1000
batch_size = 500
restore_checkpoint = False
n_iterations_per_epoch = len(X_train) // batch_size
n_iterations_validation = len(valX) // batch_size
best_loss_val = np.infty
checkpoint_path = "./my_capsule_network"

with tf.Session() as sess:
    if restore_checkpoint and tf.train.checkpoint_exists(checkpoint_path):
        saver.restore(sess, checkpoint_path)
    else:
        init.run()

    for epoch in range(n_epochs):
        for iteration in range(1, n_iterations_per_epoch + 1):
            X_batch = X_train[(iteration-1)*batch_size:(iteration*batch_size),:]
            y_batch = Y_train[(iteration-1)*batch_size:(iteration*batch_size)]
            # Run the training operation and measure the loss:
            _, loss_train = sess.run(
                [training_op, loss],
                feed_dict={X: X_batch.reshape([-1, 5, 5, 1]),
                           y: y_batch.reshape([-1]),
                           mask_with_labels: True})
            print("\rIteration: {}/{} ({:.1f}%)  Loss: {:.5f}".format(
                      iteration, n_iterations_per_epoch,
                      iteration * 100 / n_iterations_per_epoch,
                      loss_train),
                  end="")
            
        # At the end of each epoch,  
        # measure the validation loss and accuracy:
        loss_vals = []
        acc_vals = []
        for iteration in range(1, n_iterations_validation + 1):
            X_batch = valX[(iteration-1)*batch_size:(iteration*batch_size),:]
            y_batch = valY[(iteration-1)*batch_size:(iteration*batch_size)]
            loss_val, acc_val = sess.run(
                    [loss, accuracy],
                    feed_dict={X: X_batch.reshape([-1, 5, 5, 1]),
                               y: y_batch.reshape([-1])})
            loss_vals.append(loss_val)
            acc_vals.append(acc_val)
            print("\rEvaluating the model: {}/{} ({:.1f}%)".format(
                      iteration, n_iterations_validation,
                      iteration * 100 / n_iterations_validation),
                  end=" " * 10)
        loss_val = np.mean(loss_vals)
        acc_val = np.mean(acc_vals)
        print("\rEpoch: {}  Val accuracy: {:.4f}%  Loss: {:.6f}{}".format(
            epoch + 1, acc_val * 100, loss_val,
            " (improved)" if loss_val < best_loss_val else ""))

        # And save the model if it improved: 
        
        if loss_val < best_loss_val:
            save_path = saver.save(sess, checkpoint_path)
            best_loss_val = loss_val

Epoch: 1  Val accuracy: 99.8527%  Loss: 0.047239 (improved)
Epoch: 2  Val accuracy: 99.8725%  Loss: 0.035852 (improved)
Epoch: 3  Val accuracy: 99.8879%  Loss: 0.029497 (improved)
Epoch: 4  Val accuracy: 99.9121%  Loss: 0.027808 (improved)
Epoch: 5  Val accuracy: 99.9209%  Loss: 0.027297 (improved)
Epoch: 6  Val accuracy: 99.9077%  Loss: 0.027483
Epoch: 7  Val accuracy: 99.9187%  Loss: 0.026489 (improved)
Epoch: 8  Val accuracy: 99.9253%  Loss: 0.026968
Epoch: 9  Val accuracy: 99.9187%  Loss: 0.027316
Epoch: 10  Val accuracy: 99.9253%  Loss: 0.027482
Epoch: 11  Val accuracy: 99.9209%  Loss: 0.027683
Epoch: 12  Val accuracy: 99.9319%  Loss: 0.028037
Epoch: 13  Val accuracy: 99.9165%  Loss: 0.027182
Epoch: 14  Val accuracy: 99.9275%  Loss: 0.028008
Epoch: 15  Val accuracy: 99.9253%  Loss: 0.026290 (improved)
Epoch: 16  Val accuracy: 99.9231%  Loss: 0.027543
Epoch: 17  Val accuracy: 99.9363%  Loss: 0.028715
Epoch: 18  Val accuracy: 99.9231%  Loss: 0.027996
Epoch: 19  Val accuracy: 99.9143

Epoch: 161  Val accuracy: 99.9297%  Loss: 0.036110
Epoch: 162  Val accuracy: 99.9297%  Loss: 0.034336
Epoch: 163  Val accuracy: 99.9341%  Loss: 0.034813
Epoch: 164  Val accuracy: 99.9187%  Loss: 0.036823
Epoch: 165  Val accuracy: 99.9319%  Loss: 0.034217
Epoch: 166  Val accuracy: 99.9341%  Loss: 0.033772
Epoch: 167  Val accuracy: 99.9319%  Loss: 0.035242
Epoch: 168  Val accuracy: 99.9297%  Loss: 0.036157
Epoch: 169  Val accuracy: 99.9297%  Loss: 0.035891
Epoch: 170  Val accuracy: 99.9297%  Loss: 0.036367
Epoch: 171  Val accuracy: 99.9253%  Loss: 0.038890
Epoch: 172  Val accuracy: 99.9297%  Loss: 0.035324
Epoch: 173  Val accuracy: 99.9385%  Loss: 0.035722
Epoch: 174  Val accuracy: 99.9319%  Loss: 0.034998
Epoch: 175  Val accuracy: 99.9341%  Loss: 0.034417
Epoch: 176  Val accuracy: 99.9319%  Loss: 0.034053
Epoch: 177  Val accuracy: 99.9363%  Loss: 0.036743
Epoch: 178  Val accuracy: 99.9297%  Loss: 0.035736
Epoch: 179  Val accuracy: 99.9297%  Loss: 0.033762
Epoch: 180  Val accuracy: 99.93

Epoch: 321  Val accuracy: 99.9341%  Loss: 0.036867
Epoch: 322  Val accuracy: 99.9275%  Loss: 0.038523
Epoch: 323  Val accuracy: 99.9297%  Loss: 0.039067
Epoch: 324  Val accuracy: 99.9385%  Loss: 0.038724
Epoch: 325  Val accuracy: 99.9341%  Loss: 0.038530
Epoch: 326  Val accuracy: 99.9341%  Loss: 0.039078
Epoch: 327  Val accuracy: 99.9319%  Loss: 0.036746
Epoch: 328  Val accuracy: 99.9385%  Loss: 0.039579
Epoch: 329  Val accuracy: 99.9275%  Loss: 0.041558
Epoch: 330  Val accuracy: 99.9341%  Loss: 0.039157
Epoch: 331  Val accuracy: 99.9297%  Loss: 0.039231
Epoch: 332  Val accuracy: 99.9319%  Loss: 0.037469
Epoch: 333  Val accuracy: 99.9341%  Loss: 0.038346
Epoch: 334  Val accuracy: 99.9363%  Loss: 0.038570
Epoch: 335  Val accuracy: 99.9341%  Loss: 0.040412
Epoch: 336  Val accuracy: 99.9341%  Loss: 0.038228
Epoch: 337  Val accuracy: 99.9319%  Loss: 0.038619
Epoch: 338  Val accuracy: 99.9363%  Loss: 0.038655
Epoch: 339  Val accuracy: 99.9341%  Loss: 0.038401
Epoch: 340  Val accuracy: 99.93

Epoch: 481  Val accuracy: 99.9363%  Loss: 0.039224
Epoch: 482  Val accuracy: 99.9363%  Loss: 0.038980
Epoch: 483  Val accuracy: 99.9385%  Loss: 0.038611
Epoch: 484  Val accuracy: 99.9363%  Loss: 0.039284
Epoch: 485  Val accuracy: 99.9319%  Loss: 0.040248
Epoch: 486  Val accuracy: 99.9341%  Loss: 0.038343
Epoch: 487  Val accuracy: 99.9363%  Loss: 0.038107
Epoch: 488  Val accuracy: 99.9275%  Loss: 0.037566
Epoch: 489  Val accuracy: 99.9319%  Loss: 0.040605
Epoch: 490  Val accuracy: 99.9319%  Loss: 0.039858
Epoch: 491  Val accuracy: 99.9385%  Loss: 0.036680
Epoch: 492  Val accuracy: 99.9341%  Loss: 0.037393
Epoch: 493  Val accuracy: 99.9341%  Loss: 0.038399
Epoch: 494  Val accuracy: 99.9319%  Loss: 0.037795
Epoch: 495  Val accuracy: 99.9319%  Loss: 0.039772
Epoch: 496  Val accuracy: 99.9275%  Loss: 0.039098
Epoch: 497  Val accuracy: 99.9341%  Loss: 0.037530
Epoch: 498  Val accuracy: 99.9275%  Loss: 0.038658
Epoch: 499  Val accuracy: 99.9407%  Loss: 0.037146
Epoch: 500  Val accuracy: 99.93

Epoch: 641  Val accuracy: 99.9297%  Loss: 0.039444
Epoch: 642  Val accuracy: 99.9319%  Loss: 0.038456
Epoch: 643  Val accuracy: 99.9275%  Loss: 0.039294
Epoch: 644  Val accuracy: 99.9385%  Loss: 0.036837
Epoch: 645  Val accuracy: 99.9363%  Loss: 0.037664
Epoch: 646  Val accuracy: 99.9319%  Loss: 0.037914
Epoch: 647  Val accuracy: 99.9363%  Loss: 0.040378
Epoch: 648  Val accuracy: 99.9341%  Loss: 0.037630
Epoch: 649  Val accuracy: 99.9275%  Loss: 0.038450
Epoch: 650  Val accuracy: 99.9319%  Loss: 0.038251
Epoch: 651  Val accuracy: 99.9363%  Loss: 0.037828
Epoch: 652  Val accuracy: 99.9319%  Loss: 0.038770
Epoch: 653  Val accuracy: 99.9385%  Loss: 0.038123
Epoch: 654  Val accuracy: 99.9363%  Loss: 0.037950
Epoch: 655  Val accuracy: 99.9341%  Loss: 0.038017
Epoch: 656  Val accuracy: 99.9341%  Loss: 0.041948
Epoch: 657  Val accuracy: 99.9363%  Loss: 0.042409
Epoch: 658  Val accuracy: 99.9319%  Loss: 0.039877
Epoch: 659  Val accuracy: 99.9319%  Loss: 0.038254
Epoch: 660  Val accuracy: 99.93

Epoch: 801  Val accuracy: 99.9407%  Loss: 0.039491
Epoch: 802  Val accuracy: 99.9363%  Loss: 0.039592
Epoch: 803  Val accuracy: 99.9297%  Loss: 0.042099
Epoch: 804  Val accuracy: 99.9341%  Loss: 0.038291
Epoch: 805  Val accuracy: 99.9297%  Loss: 0.037212
Epoch: 806  Val accuracy: 99.9407%  Loss: 0.037920
Epoch: 807  Val accuracy: 99.9319%  Loss: 0.038807
Epoch: 808  Val accuracy: 99.9363%  Loss: 0.039641
Epoch: 809  Val accuracy: 99.9341%  Loss: 0.038180
Epoch: 810  Val accuracy: 99.9385%  Loss: 0.039144
Epoch: 811  Val accuracy: 99.9363%  Loss: 0.039216
Epoch: 812  Val accuracy: 99.9385%  Loss: 0.040003
Epoch: 813  Val accuracy: 99.9341%  Loss: 0.040175
Epoch: 814  Val accuracy: 99.9341%  Loss: 0.039638
Epoch: 815  Val accuracy: 99.9363%  Loss: 0.039777
Epoch: 816  Val accuracy: 99.9385%  Loss: 0.039978
Epoch: 817  Val accuracy: 99.9341%  Loss: 0.047559
Epoch: 818  Val accuracy: 99.9385%  Loss: 0.037374
Epoch: 819  Val accuracy: 99.9341%  Loss: 0.039890
Epoch: 820  Val accuracy: 99.93

Epoch: 961  Val accuracy: 99.9363%  Loss: 0.039876
Epoch: 962  Val accuracy: 99.9319%  Loss: 0.040465
Epoch: 963  Val accuracy: 99.9341%  Loss: 0.040126
Epoch: 964  Val accuracy: 99.9319%  Loss: 0.044136
Epoch: 965  Val accuracy: 99.9385%  Loss: 0.040312
Epoch: 966  Val accuracy: 99.9363%  Loss: 0.038733
Epoch: 967  Val accuracy: 99.9407%  Loss: 0.038550
Epoch: 968  Val accuracy: 99.9363%  Loss: 0.038896
Epoch: 969  Val accuracy: 99.9341%  Loss: 0.039369
Epoch: 970  Val accuracy: 99.9363%  Loss: 0.039171
Epoch: 971  Val accuracy: 99.9341%  Loss: 0.039845
Epoch: 972  Val accuracy: 99.9363%  Loss: 0.040105
Epoch: 973  Val accuracy: 99.9341%  Loss: 0.044983
Epoch: 974  Val accuracy: 99.9363%  Loss: 0.040185
Epoch: 975  Val accuracy: 99.9407%  Loss: 0.038938
Epoch: 976  Val accuracy: 99.9363%  Loss: 0.039111
Epoch: 977  Val accuracy: 99.9385%  Loss: 0.040539
Epoch: 978  Val accuracy: 99.9385%  Loss: 0.039333
Epoch: 979  Val accuracy: 99.9363%  Loss: 0.041821
Epoch: 980  Val accuracy: 99.93

# Testing

In [6]:
n_iterations_test = len(testX) // batch_size
checkpoint_path = "./my_capsule_network"
with tf.Session() as sess:
    saver.restore(sess, checkpoint_path)
    pred = []
    loss_tests = []
    acc_tests = []
    for iteration in range(1, n_iterations_test + 1):
        X_batch = testX[(iteration-1)*batch_size:(iteration*batch_size),:]
        y_batch = testY[(iteration-1)*batch_size:(iteration*batch_size)]
        loss_test, acc_test = sess.run(
                [loss, accuracy],
                feed_dict={X: X_batch.reshape([-1, 5, 5, 1]),
                           y: y_batch.reshape([-1])})
        loss_tests.append(loss_test)
        pred.append(y_pred)
        acc_tests.append(acc_test)
        print("\rEvaluating the model: {}/{} ({:.1f}%)".format(
                  iteration, n_iterations_test,
                  iteration * 100 / n_iterations_test),
              end=" " * 10)
    loss_test = np.mean(loss_tests)
    #print(tf.confusion_matrix())
    acc_test = np.mean(acc_tests)
    print("\rFinal test accuracy: {:.4f}%  Loss: {:.6f}".format(
        acc_test * 100, loss_test))


INFO:tensorflow:Restoring parameters from ./my_capsule_network
Final test accuracy: 99.9345%  Loss: 0.016660   


# Training Set Predictions

In [7]:
with tf.Session() as sess:
    saver.restore(sess, checkpoint_path)
    y_pred_value = sess.run(
            [y_pred],
            feed_dict={X: X_train.reshape([-1, 5, 5, 1]),
                       y: np.array([], dtype=np.int64)})
pred = np.array(y_pred_value).T
pred = pred.flatten()
Y_train = Y_train.flatten()

from sklearn.metrics import classification_report,confusion_matrix
print("")
print("Confusion Matrix is :")
print(confusion_matrix(Y_train, pred, labels=None, sample_weight=None))
print("")
print("Classification report is : ")
print("")
print(classification_report(Y_train, pred, labels=None, sample_weight=None))

INFO:tensorflow:Restoring parameters from ./my_capsule_network

Confusion Matrix is :
[[181939     22]
 [    64    251]]

Classification report is : 

             precision    recall  f1-score   support

          0       1.00      1.00      1.00    181961
          1       0.92      0.80      0.85       315

avg / total       1.00      1.00      1.00    182276



# Validation set predictions

In [8]:
with tf.Session() as sess:
    saver.restore(sess, checkpoint_path)
    y_pred_value = sess.run(
            [y_pred],
            feed_dict={X: valX.reshape([-1, 5, 5, 1]),
                       y: np.array([], dtype=np.int64)})
pred = np.array(y_pred_value).T
pred = pred.flatten()
valY = valY.flatten()

from sklearn.metrics import classification_report,confusion_matrix
print("")
print("Confusion Matrix is :")
print(confusion_matrix(valY, pred, labels=None, sample_weight=None))
print("")
print("Classification report is : ")
print("")
print(classification_report(valY, pred, labels=None, sample_weight=None))

INFO:tensorflow:Restoring parameters from ./my_capsule_network

Confusion Matrix is :
[[45486     4]
 [   26    53]]

Classification report is : 

             precision    recall  f1-score   support

          0       1.00      1.00      1.00     45490
          1       0.93      0.67      0.78        79

avg / total       1.00      1.00      1.00     45569



# Test Set Predictions

In [9]:
with tf.Session() as sess:
    saver.restore(sess, checkpoint_path)
    y_pred_value = sess.run(
            [y_pred],
            feed_dict={X: testX.reshape([-1, 5, 5, 1]),
                       y: np.array([], dtype=np.int64)})
pred = np.array(y_pred_value).T
pred = pred.flatten()
testY = testY.flatten()

from sklearn.metrics import classification_report,confusion_matrix
print("")
print("Confusion Matrix is :")
print(confusion_matrix(testY, pred, labels=None, sample_weight=None))
print("")
print("Classification report is : ")
print("")
print(classification_report(testY, pred, labels=None, sample_weight=None))

INFO:tensorflow:Restoring parameters from ./my_capsule_network

Confusion Matrix is :
[[56851    13]
 [   24    74]]

Classification report is : 

             precision    recall  f1-score   support

          0       1.00      1.00      1.00     56864
          1       0.85      0.76      0.80        98

avg / total       1.00      1.00      1.00     56962

